# Bone Fracture Detection using YOLOv8

This Jupyter Notebook explores the application of deep learning for bone fracture detection using a comprehensive X-ray image dataset.  The dataset is specifically designed for computer vision projects and aims to facilitate the development and evaluation of automated bone fracture detection algorithms.

## About the Dataset

The dataset encompasses X-ray images categorized into several classes, each representing a specific type of bone fracture within the upper extremities. These classes include:

*   Elbow Positive
*   Fingers Positive
*   Forearm Fracture
*   Humerus Fracture
*   Shoulder Fracture
*   Wrist Positive

Each image is annotated with either bounding boxes or pixel-level segmentation masks, precisely indicating the location and extent of the detected fracture. These annotations are crucial for training and evaluating bone fracture detection algorithms, particularly object detection models.

This dataset provides a valuable resource for researchers and developers working on automated fracture detection. Its diverse range of fracture classes enables the training of robust models capable of accurately identifying fractures in various regions of the upper extremities. The ultimate goal of this dataset is to accelerate the development of computer vision solutions for automated fracture detection, thereby contributing to advancements in medical diagnostics and improved patient care.

**When using this dataset for your research, please cite it using the following DOI:** 10.13140/RG.2.2.14400.34569

**You can also find the dataset on ResearchGate:** [https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project](https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project)

In [ ]:
train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

## Imports

In [26]:

from dotenv import load_dotenv

import tensorflow as tf
from tensorflow import keras
import cv2

tf.config.list_physical_devices('GPU'), tf.__version__

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.18.0')

## Download datasets

In [27]:
# Loading kaggle keys
load_dotenv()

True

In [28]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [29]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

datasets/BoneFractureYolo8/train/


## Loading Images

In [30]:
CLASS_NAMES = ['elbow positive',
               'fingers positive',
               'forearm fracture',
               'humerus fracture',
               'humerus',
               'shoulder fracture',
               'wrist positive']

class_mapping = dict(zip(range(len(CLASS_NAMES)), CLASS_NAMES))
class_mapping

{0: 'elbow positive',
 1: 'fingers positive',
 2: 'forearm fracture',
 3: 'humerus fracture',
 4: 'humerus',
 5: 'shoulder fracture',
 6: 'wrist positive'}

In [31]:
TRAIN_DIR = cfg.DATASET_DIRS.TRAIN_DIR
VALIDATION_DIR = cfg.DATASET_DIRS.VALIDATION_DIR
TEST_DIR = cfg.DATASET_DIRS.TEST_DIR

TRAIN_IMAGE_DIR = f'{TRAIN_DIR}/images'
TRAIN_LABELS_DIR = f'{TRAIN_DIR}/labels'

VALID_IMAGE_DIR = f'{VALIDATION_DIR}/images'
VALID_LABELS_DIR = f'{VALIDATION_DIR}/labels'

TEST_IMAGE = f'{TEST_DIR}/images'
TEST_LABELS = f'{TEST_DIR}/labels'

IMG_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE

### Training and Validation Dataset setup

In [32]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return tf.cast(image, tf.float32)


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    print(bbox)
    return {"images": image, "bounding_boxes": bounding_boxes}

In [33]:
import os

import numpy as np
import tqdm
from tqdm.notebook import tqdm


class PrepareDataset:
    def __init__(self, image_dir: str, label_dir: str) -> None:
        """
        Args:
            image_dir (str): Path to the directory containing the images.
            label_dir (str): Path to the directory containing the labels.
        """
        self.image_dir = image_dir
        self.label_dir = label_dir

    def get_dataset(self) -> tuple[list[str], list[int], list[np.ndarray]]:
        """Loads and parses YOLOv8 labels.

        Args:
            None

        Returns:
            tuple[list[str], list[int], list[np.ndarray]]: A tuple containing:
                - A list of image paths.
                - A list of class ids.
                - A list of bounding boxes, where each bounding box is an array of 8 floats.
        """
        image_paths = []
        class_ids = []
        bboxes = []
        for file_name in tqdm(os.listdir(self.image_dir)):
            if file_name.endswith((".jpg", ".png")):
                imaga_file_path = os.path.join(self.image_dir, file_name)
                label_file_path = os.path.join(self.label_dir, file_name.replace(
                    ".jpg", ".txt").replace(".png", ".txt"))

                if not os.path.exists(label_file_path):
                    print(f"Label file not found for image: {imaga_file_path}")
                    continue

                with open(label_file_path, 'r') as f:
                    lines = f.readlines()

                if not lines:
                    continue

                image_bboxes = []
                image_classes = []


                # 0 0.458736435546875 0.3806510419921875 0.3614540244140625 0.389472591796875 0.36892872265625 0.48237111328125 0.457112263671875 0.471341630859375 0.458736435546875 0.3806510419921875
                for line in lines:
                    try:
                        values = [float(value) for value in line.split()]
                        class_id = int(values[0])
                        coords = np.array(values[1:5])  # Coords are already normalized YOLO format
                        # coords = np.array(values[1:])  # Coords are already normalized YOLO format
                        # Reshape coords to (1, 4) if it's a single box
                        if coords.size == 4:
                            coords = coords.reshape(1, 4)  # Ensure it's a 2D array even for one box

                        image_bboxes.append(coords)
                        image_classes.append(class_id)

                    except Exception as e:
                        print(f"[ERROR] - {e} in file {label_file_path} on line: {line}")
                        continue
                    
                image_paths.append(imaga_file_path)
                class_ids.append(np.array(image_classes))
                bboxes.append(np.concatenate(image_bboxes, axis=0))  # Concatenate boxes for the image

        return image_paths, class_ids, bboxes


In [34]:
# from utils.prepare_dataset import PrepareDataset

preparer_train_ds = PrepareDataset(image_dir=TRAIN_IMAGE_DIR,
                                   label_dir=TRAIN_LABELS_DIR)
image_paths, class_ids, bboxes = preparer_train_ds.get_dataset()
len(image_paths), len(class_ids), len(bboxes)

  0%|          | 0/3631 [00:00<?, ?it/s]

(1804, 1804, 1804)

In [35]:
class_ids

[array([2]),
 array([2]),
 array([0]),
 array([1]),
 array([6]),
 array([2]),
 array([1]),
 array([5]),
 array([4]),
 array([4]),
 array([1]),
 array([6, 6]),
 array([2]),
 array([4, 4]),
 array([1]),
 array([1]),
 array([4, 3]),
 array([0]),
 array([0]),
 array([5]),
 array([6, 6]),
 array([1]),
 array([0]),
 array([1]),
 array([4]),
 array([2]),
 array([4]),
 array([6]),
 array([6, 6]),
 array([2]),
 array([4]),
 array([5]),
 array([0]),
 array([1]),
 array([5]),
 array([1, 1]),
 array([5]),
 array([2]),
 array([0]),
 array([4]),
 array([0]),
 array([2]),
 array([2]),
 array([0]),
 array([5, 5]),
 array([1]),
 array([4, 4]),
 array([1, 1, 1]),
 array([5]),
 array([4]),
 array([1]),
 array([2]),
 array([1, 1]),
 array([5]),
 array([1]),
 array([1]),
 array([2]),
 array([4]),
 array([1]),
 array([4]),
 array([6]),
 array([0]),
 array([5]),
 array([1]),
 array([4]),
 array([6, 6]),
 array([2]),
 array([0]),
 array([5]),
 array([1]),
 array([1]),
 array([5]),
 array([1]),
 array([1, 1]),

In [36]:
bboxes

[array([[0.67095098, 0.17080413, 0.54633244, 0.17960875]]),
 array([[0.54937497, 0.45355966, 0.42703269, 0.41726112]]),
 array([[0.59830333, 0.56517056, 0.50202929, 0.58381238]]),
 array([[0.70736082, 0.52222953, 0.66558816, 0.44930158]]),
 array([[0.68223918, 0.63961631, 0.39150739, 0.59077927]]),
 array([[0.67270002, 0.37535682, 0.5281243 , 0.37814663]]),
 array([[0.64857493, 0.41632891, 0.60150954, 0.40525873]]),
 array([[0.80460021, 0.6527411 , 0.71474379, 0.68913262]]),
 array([[0.49255314, 0.58046875, 0.645986  , 0.51738817]]),
 array([[0.50880782, 0.20323453, 0.34333329, 0.2015625 ]]),
 array([[0.68522163, 0.44765625, 0.65095985, 0.51113633]]),
 array([[0.38865041, 0.56089771, 0.48520196, 0.52806021],
        [0.5405881 , 0.63003763, 0.43044275, 0.62373755]]),
 array([[0.30837434, 0.4125    , 0.38893433, 0.39547779]]),
 array([[0.63707473, 0.37305553, 0.48875068, 0.27303412],
        [0.48377458, 0.56366745, 0.32412773, 0.4986977 ]]),
 array([[0.50207057, 0.30643779, 0.56627211,

In [37]:
bboxes_tensor = tf.ragged.constant(bboxes)
classes_tensor = tf.ragged.constant(class_ids)
image_paths_tensor = tf.ragged.constant(image_paths)
bboxes_tensor, classes_tensor, image_paths_tensor

(<tf.RaggedTensor [[[0.6709509778325123, 0.170804130859375, 0.5463324359605911,
    0.179608748046875]]                                       ,
  [[0.5493749660493827, 0.453559662109375, 0.42703269135802463,
    0.41726112109375]]                                         ,
  [[0.5983033349753695, 0.5651705625, 0.5020292881773399, 0.58381237890625]],
  ..., [[0.4901534219858156, 0.454840611328125, 0.27035615957446807,
         0.507783462890625]]                                        ,
  [[0.5512980416666666, 0.58523765625, 0.7055332037037038, 0.6066540546875]],
  [[0.32651640721649483, 0.576287625, 0.4993145721649484, 0.610761435546875]]]>,
 <tf.RaggedTensor [[2], [2], [0], ..., [0], [2], [1]]>,
 <tf.Tensor: shape=(1804,), dtype=string, numpy=
 array([b'datasets/BoneFractureYolo8/train//images/image1_1511_png.rf.2056dd62da1e447607536611c901c3f0.jpg',
        b'datasets/BoneFractureYolo8/train//images/image1_71_png.rf.53f6dcd7657036e92bbb2353a848eb99.jpg',
        b'datasets/BoneFractur

In [38]:
train_data = tf.data.Dataset.from_tensor_slices(
    (image_paths_tensor, classes_tensor, bboxes_tensor))

In [39]:
train_data

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), RaggedTensorSpec(TensorShape([None]), tf.int32, 0, tf.int64), RaggedTensorSpec(TensorShape([None, None]), tf.float64, 1, tf.int64))>

### Augmentation

In [40]:
import keras_cv

resizing_layer = keras_cv.layers.JitteredResize(
            target_size=(IMG_SIZE, IMG_SIZE), scale_factor=(0.75, 1.3), bounding_box_format="CENTER_XYWH"
        )
augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="CENTER_XYWH"),
        keras_cv.layers.RandomShear(
            x_factor=0.2, y_factor=0.2, bounding_box_format="CENTER_XYWH"
        ),
        resizing_layer
    ]
)



In [41]:
train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE).shuffle(BATCH_SIZE * 4).ragged_batch(BATCH_SIZE, drop_remainder=True).map(resizing, num_parallel_calls=tf.data.AUTOTUNE)


tf.RaggedTensor(values=Tensor("RaggedFromVariant_1/RaggedTensorFromVariant:1", shape=(None,), dtype=float64), row_splits=Tensor("RaggedFromVariant_1/RaggedTensorFromVariant:0", shape=(None,), dtype=int64))


In [42]:


# train_ds = train_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)

In [43]:
inputs = next(iter(train_ds.take(1)))
inputs["images"], inputs["bounding_boxes"]

2025-02-10 12:03:49.701846: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: CANCELLED: Loop execution was cancelled.
	 [[{{function_node jittered_resize_1_1_map_while_body_2149}}{{node jittered_resize_1_1/map/while/map/while}}]]


(<tf.Tensor: shape=(16, 224, 224, 3), dtype=float32, numpy=
 array([[[[ 10.       ,  10.       ,  10.       ],
          [ 10.       ,  10.       ,  10.       ],
          [ 10.       ,  10.       ,  10.       ],
          ...,
          [  6.       ,   6.       ,   6.       ],
          [  6.       ,   6.       ,   6.       ],
          [  6.       ,   6.       ,   6.       ]],
 
         [[ 10.       ,  10.       ,  10.       ],
          [ 10.       ,  10.       ,  10.       ],
          [ 10.       ,  10.       ,  10.       ],
          ...,
          [  6.       ,   6.       ,   6.       ],
          [  6.       ,   6.       ,   6.       ],
          [  6.       ,   6.       ,   6.       ]],
 
         [[ 10.       ,  10.       ,  10.       ],
          [ 10.       ,  10.       ,  10.       ],
          [ 10.       ,  10.       ,  10.       ],
          ...,
          [  6.       ,   6.       ,   6.       ],
          [  6.       ,   6.       ,   6.       ],
          [  6.       

In [44]:
valid_datasets = PrepareDataset(image_dir=VALID_IMAGE_DIR,
                                label_dir=VALID_LABELS_DIR)
valid_image_paths, valid_labels, valid_bboxes = valid_datasets.get_dataset()
len(valid_image_paths), len(valid_labels), len(valid_bboxes)

  0%|          | 0/348 [00:00<?, ?it/s]

(173, 173, 173)

In [45]:

val_data = tf.data.Dataset.from_tensor_slices((tf.ragged.constant(valid_image_paths),
                                               tf.ragged.constant(valid_labels),
                                               tf.ragged.constant(valid_bboxes)))
val_ds = val_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(BATCH_SIZE * 4)
val_ds = val_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)

tf.RaggedTensor(values=Tensor("RaggedFromVariant_1/RaggedTensorFromVariant:1", shape=(None,), dtype=float64), row_splits=Tensor("RaggedFromVariant_1/RaggedTensorFromVariant:0", shape=(None,), dtype=int64))


In [46]:
resizing = keras_cv.layers.JitteredResize(
    target_size=(IMG_SIZE, IMG_SIZE),
    scale_factor=(0.75, 1.3),
    bounding_box_format="CENTER_XYWH",
)
val_ds = val_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)

In [47]:
def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]

In [48]:
train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [49]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_s_backbone_coco"  # We will use yolov8 small backbone with coco weights
)
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="CENTER_XYWH",
    backbone=backbone,
    fpn_depth=1,
)

ImportError: `from_preset()` requires the `kagglehub` package. Please install with `pip install kagglehub`.